In [1]:
from dash import Dash
import dash_leaflet as dl

from database import Base, FeatureSet, Feature, Colormap, Style, connect_db
from shapely.wkb import loads
from shapely.geometry import shape, mapping

In [2]:
def create_marker(position, text, popup) -> dl.Marker:
    return dl.Marker(position=position, children=[
        dl.Tooltip(text),
        dl.Popup(popup)
    ])

In [3]:
def create_geojson(feature) -> dl.GeoJSON:

    return dl.GeoJSON(data=feature)

In [7]:
def feature_to_geojson(feature):
    """
    Convert a Feature from the database to a GeoJSON Feature object
    """

    properties = feature.properties
    geometry_type = feature.geometry_type
    feature_set = feature.feature_set

    raw_geometry = feature.geometry.data
    shape_geometry = loads(bytes(raw_geometry))
    geojson_geometry = mapping(shape_geometry)

    print(geojson_geometry)

    geojson =  {
        "type": "Feature",
        "geometry": geojson_geometry,
        "properties": properties
    }

    return geojson

In [9]:
engine, session = connect_db()

# get the first Feature of the feature set with the id 10
feature = session.query(Feature).filter(Feature.feature_set_id == 10).first()
feature = feature_to_geojson(feature)
feature = create_geojson(feature)
feature

{'type': 'MultiPolygon', 'coordinates': [(((10.02821890497073, 53.53700066405299), (10.02893827739822, 53.53671991850927), (10.028938175253131, 53.5367198753419), (10.02886288308047, 53.5365910657264), (10.028720086153651, 53.53634676763295), (10.028772522720699, 53.536335881130306), (10.028822253787434, 53.536325555427034), (10.028786597557172, 53.53626466211497), (10.028757401480592, 53.53621480209454), (10.028441877669223, 53.53567600380371), (10.02841602107748, 53.53563185088368), (10.028325694782037, 53.53562172399803), (10.028233713849604, 53.53568342472628), (10.028153744387211, 53.535737066802255), (10.02807371503263, 53.53579073630552), (10.027993641089031, 53.53584444209347), (10.02791353698888, 53.535898157074804), (10.027833418253227, 53.53595189909415), (10.027592941883182, 53.53611319776741), (10.027512747776818, 53.536166976170676), (10.027432583856998, 53.53622076324858), (10.02719197084353, 53.53638215216093), (10.027071730233448, 53.53646279193102), (10.02699683144335

GeoJSON(data={'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [(((10.02821890497073, 53.53700066405299), (10.02893827739822, 53.53671991850927), (10.028938175253131, 53.5367198753419), (10.02886288308047, 53.5365910657264), (10.028720086153651, 53.53634676763295), (10.028772522720699, 53.536335881130306), (10.028822253787434, 53.536325555427034), (10.028786597557172, 53.53626466211497), (10.028757401480592, 53.53621480209454), (10.028441877669223, 53.53567600380371), (10.02841602107748, 53.53563185088368), (10.028325694782037, 53.53562172399803), (10.028233713849604, 53.53568342472628), (10.028153744387211, 53.535737066802255), (10.02807371503263, 53.53579073630552), (10.027993641089031, 53.53584444209347), (10.02791353698888, 53.535898157074804), (10.027833418253227, 53.53595189909415), (10.027592941883182, 53.53611319776741), (10.027512747776818, 53.536166976170676), (10.027432583856998, 53.53622076324858), (10.02719197084353, 53.53638215216093), (10.0270717302

In [10]:
app = Dash()

app.layout = dl.Map(
    [dl.TileLayer()], # defaults to OpenStreetMap
    center=[53.55, 9.99],
    zoom=12,
    style={'width': '1000px', 'height': '500px'},
)

marker = create_marker(position=[53.55, 9.99], text='Hamburg', popup='The City of Hamburg')
app.layout.children.append(marker)
app.layout.children.append(feature)

if __name__ == '__main__':
    app.run_server(debug=True)